In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import re

# Eda on the toxic classification data

In [3]:
df= pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/train.csv')
df.head()

In [4]:
df[df['identity_hate']==1]

In [5]:
df.loc[:, 'toxic':'identity_hate'].sum(axis=0).plot(kind='bar')

In [6]:
# distribution of data
df.loc[:, 'toxic':'identity_hate'].sum(axis=1).hist()

In [7]:
# wietghting the categories
df['severe_toxic'] = df.severe_toxic *3
df['threat'] = df.threat *2
'''
df['severe_toxic'] = df.severe_toxic +9
df['obscene'] = df.obscene +9
df['threat'] = df.threat +9
df['insult'] = df.insult +9
df['identity_hate'] = df.identity_hate +9 
'''

In [8]:
# adding score column
df['score']= df.loc[:, 'toxic':'identity_hate'].sum(axis=1)
df['score']= df['score']+1
data= df[['comment_text', 'score']]

In [9]:
data.head()

In [10]:
# distribution of data
df['score'].hist()

In [11]:
# number of values for each score
data['score'].value_counts()

In [12]:

# samling from the neutral exaples
data = pd.concat([data[data.score>1] , 
                data[data.score==1].sample(int(len(data[data.score>1])*1.5)) ], axis=0).sample(frac=1)

print(data.shape)


In [13]:
# number of values for each score
data['score'].value_counts()

In [14]:
# distribution of data
data['score'].hist()

## Feature scaling scores

In [15]:
'''
from sklearn import preprocessing

x = data['score'].values.reshape(-1, 1)#returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
data['score_scaled']= x_scaled
'''

In [16]:
#data.head()

In [17]:
'''
print(data[data['score_scaled']==1].head())
print(data[data['score_scaled']==0].head())
'''

In [18]:
'''
print(data[data['score_scaled']>0.2].head())
'''

In [19]:
'''
# distribution of data
data['score_scaled'].hist()
'''

In [20]:
'''
# number of values for each score
data['score_scaled'].value_counts()
'''

# Eda on ruddit data

In [21]:
df_ = pd.read_csv("../input/ruddit-jigsaw-dataset/Dataset/ruddit_with_text.csv")
print(df_.shape)

In [22]:
df_.head()

In [23]:
ruddit_data= df_[['txt', 'offensiveness_score']]

In [24]:
ruddit_data.head()

In [25]:
# distribution of data
ruddit_data['offensiveness_score'].hist()

In [26]:
ruddit_data[ruddit_data['offensiveness_score'] >0.9]

In [27]:
ruddit_data['txt'][245]

# Preparing data for Ridge pipeline

In [28]:
# stopwards
from nltk.corpus import stopwords
stop = stopwords.words('english')
stop[0:5]

In [29]:
def clean_text(c):
    c = re.sub("[^a-zA-Z0-9\\s]", "", c)
    c = re.sub('\n', " ", c)
    c = re.sub("\d", "", c)
    return c

In [30]:
def clean(df, column):
    df[column]= df[column].apply(lambda c: clean_text(c))
    df[column]= df[column].str.lower()
    
    # Removing stop words
    pat = r'\b(?:{})\b'.format('|'.join(stop))
    df[column] = df[column].str.replace(pat, '')
    df[column] = df[column].str.replace(r'\s+', ' ')
    
    return df 

In [31]:
data= clean(data, 'comment_text')

In [32]:
data.head()

# Preparing ruddit data

In [33]:
ruddit_data= clean(ruddit_data, 'txt')

In [34]:
ruddit_data.head()

# Modeling

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.pipeline import Pipeline

In [36]:
pipeline = Pipeline([('vec', TfidfVectorizer(min_df= 3, max_df=0.5, analyzer = 'char_wb',stop_words={'english'}, ngram_range = (3,5))),
                   ("clf", Ridge(alpha= 0.8))])

In [37]:
pipeline.fit(data['comment_text'], data['score'])

# Modeling 2

In [74]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


In [110]:
vocab_size = 20000
embedding_dim = 8
max_length = 300
trunc_type='post'
oov_tok = "<OOV>"


In [111]:
# training data
text= data['comment_text']
score= data['score']

toxic_data_tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
toxic_data_tokenizer.fit_on_texts(text)

In [112]:
word_index = toxic_data_tokenizer.word_index
#word_index

In [113]:
toxic_data_sequences = toxic_data_tokenizer.texts_to_sequences(text)
toxic_data_padded = pad_sequences(toxic_data_sequences,maxlen=max_length, truncating=trunc_type)
toxic_data_padded

In [114]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])
model.compile(loss='mse',optimizer='adam',metrics=['mse'])
model.summary()

In [116]:
num_epochs = 2

model.fit(toxic_data_padded, score,  epochs=num_epochs)

# Ruddit data pipline

In [46]:
ruddit_pl = Pipeline([('vec', TfidfVectorizer(min_df= 3, max_df=0.5, analyzer = 'char_wb',stop_words={'english'}, ngram_range = (3,5))),
                   ("clf", Ridge(alpha= 0.8))])

In [47]:
ruddit_pl.fit(ruddit_data['txt'], ruddit_data['offensiveness_score'])

# Modeling 2 for ruddit data

In [117]:
df_train, df_test = train_test_split(ruddit_data, test_size=0.1)
print("TRAIN size:", df_train.shape)
print("TEST size:", df_test.shape)


In [118]:
# training data
ruddit_text= df_train['txt']
ruddit_score= df_train['offensiveness_score']

# test_data
ruddit_test_text= df_test['txt']
ruddit_test_score= df_test['offensiveness_score']


In [119]:
ruddit_text_tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
ruddit_text_tokenizer.fit_on_texts(ruddit_text)

In [120]:
ruddit_text_word_index = ruddit_text_tokenizer.word_index
#word_index

In [123]:
ruddit_text_sequences = ruddit_text_tokenizer.texts_to_sequences(ruddit_text)
ruddit_text_padded = pad_sequences(ruddit_text_sequences,maxlen=max_length, truncating=trunc_type)
ruddit_text_padded

In [124]:
ruddit_text_testing_sequences = ruddit_text_tokenizer.texts_to_sequences(ruddit_test_text)
ruddit_text_testing_padded = pad_sequences( ruddit_text_testing_sequences,maxlen=max_length)
ruddit_text_testing_padded


In [130]:
ruddit_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])
ruddit_model.compile(loss='mse',optimizer='adam',metrics=['mse'])
ruddit_model.summary()

In [131]:
num_epochs = 2

ruddit_model.fit(ruddit_text_padded, ruddit_score,validation_data= (ruddit_text_testing_padded, ruddit_test_score), epochs=num_epochs)

# Validation

In [69]:
df_val = pd.read_csv("../input/jigsaw-toxic-severity-rating/validation_data.csv")
print(df_val.shape)
df_val.head()

In [70]:
df_val= clean(df_val, 'less_toxic')
df_val= clean(df_val, 'more_toxic')

In [71]:
df_val.head()

## Tokenizing val text

In [82]:
val_less_sequences = tokenizer.texts_to_sequences(df_val['less_toxic'])
val_less_padded = pad_sequences(val_less_sequences,maxlen=max_length)
val_less_padded


In [83]:
val_more_sequences = tokenizer.texts_to_sequences(df_val['more_toxic'])
val_more_padded = pad_sequences(val_more_sequences,maxlen=max_length)
val_more_padded


In [133]:
# Ruddit toxenizer for validation
ruddit_text_val_sequences_less = ruddit_text_tokenizer.texts_to_sequences(df_val['less_toxic'])
ruddit_text_val_padded_less = pad_sequences(ruddit_text_val_sequences_less,maxlen=max_length, truncating=trunc_type)
ruddit_text_val_padded_less

In [134]:
# Ruddit toxenizer for validation
ruddit_text_val_sequences_more = ruddit_text_tokenizer.texts_to_sequences(df_val['more_toxic'])
ruddit_text_val_padded_more = pad_sequences(ruddit_text_val_sequences_more,maxlen=max_length, truncating=trunc_type)
ruddit_text_val_padded_more

## Predictions on validation

In [72]:
p1 = pipeline.predict(df_val['less_toxic'])
p2 = pipeline.predict(df_val['more_toxic'])

In [73]:
p3 = ruddit_pl.predict(df_val['less_toxic'])
p4 = ruddit_pl.predict(df_val['more_toxic'])

In [84]:
p5 = model.predict(val_less_padded)
p6 = model.predict(val_more_padded)

In [135]:
p7 = ruddit_model.predict(ruddit_text_val_padded_less)
p8 = ruddit_model.predict(ruddit_text_val_padded_more)

In [86]:
print(p1.shape)
print(p2.shape)
print(p3.shape)
print(p4.shape)
print(p5.shape)
print(p6.shape)
print(p7.shape)
print(p8.shape)

In [87]:
p5= p5.reshape((len(p3),))
p6= p6.reshape((len(p3),))
p7= p7.reshape((len(p3),))
p8= p8.reshape((len(p3),))

In [88]:
print(p1.shape)
print(p2.shape)
print(p3.shape)
print(p4.shape)
print(p5.shape)
print(p6.shape)
print(p7.shape)
print(p8.shape)

In [49]:
f'Validation Accuracy from Model 1 is { np.round((p1 < p2).mean() * 100,2)}'


In [50]:
f'Validation Accuracy from Model 2 is { np.round((p3 < p4).mean() * 100,2)}'

In [91]:
f'Validation Accuracy from Model 3 is { np.round((p5 < p6).mean() * 100,2)}'


In [136]:
f'Validation Accuracy from Model 4 is { np.round((p7 < p8).mean() * 100,2)}'


# ensemble

In [52]:
import scipy.optimize as optimize
import time

In [53]:
def timer(func):
    def wrapper(*args, **kws):
        st = time.time()
        res = func(*args, **kws)
        et = time.time()
        tt = (et-st)/60
        print(f'Time taken is {tt:.2f} mins')
        return res
    return wrapper

In [54]:
def optimize_wts():
    func = lambda x: -1*(((x[0]*p1 + x[1]*p3 ) < (x[0]*p2 + x[1]*p4)).mean())
    

    rranges = (slice(0.30, 0.6, 0.015), 
               slice(0.05, 0.5, 0.015)
              )
    

    resbrute = optimize.brute(func, 
                              rranges, 
                              #args=params, 
                              full_output=True,
                              finish=None)
    return resbrute
resbrute = optimize_wts()

print(resbrute[0])  # global minimum
print(resbrute[1]*-1)  # function value at global minimum

In [55]:
w1,w2= resbrute[0]
#print(best_wts)

In [64]:
p1_wt = w1*p1 + w2*p3 
p2_wt = w1*p2 + w2*p4 

In [65]:
p1_wt_= (p1+p3)/2
p2_wt_= (p2+p4)/2


In [66]:
f'Validation Accuracy from Model ensembel is { np.round((p1_wt < p2_wt).mean() * 100,2)}'

In [67]:
f'Validation Accuracy from Model ensembel is { np.round((p1_wt_ < p2_wt_).mean() * 100,2)}'

# Test

In [68]:
df_sub = pd.read_csv("../input/jigsaw-toxic-severity-rating/comments_to_score.csv")

In [69]:
df_sub.head()

In [70]:
df_sub= clean(df_sub, 'text')

In [71]:
jigsaw_pred = pipeline.predict(df_sub['text'])
ruddit_pred = ruddit_pl.predict(df_sub['text'])

In [72]:
df_sub['score'] = w1 * jigsaw_pred + w2 * ruddit_pred

In [73]:
df_sub.head()

In [74]:
# Cases with duplicates scores

df_sub['score'].count() - df_sub['score'].nunique()

In [75]:
import scipy

In [76]:
# Rank the predictions 

df_sub['score']  = scipy.stats.rankdata(df_sub['score'], method='ordinal')

print(df_sub['score'].rank().nunique())

In [77]:
df_sub[['comment_id', 'score']].to_csv("submission.csv", index=False)

In [78]:
test = pd.read_csv('../input/jigsaw-toxic-severity-rating/comments_to_score.csv')
test['score'] = df_sub.score.tolist()

In [79]:
test.sort_values('score',ascending=False).head(20)

In [80]:
test['text'][2733]

In [81]:
test.sort_values('score',ascending=True).head(20)